In [59]:
#PigLab1NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

### Step 01: Define the file paths and Load the data into Pandas dataframes and Clean

In [60]:
# Define the file paths
files = ['Lab1/DECIDE-BIOFAR.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [61]:
df

,Id,Provincia,Fecha,Patógeno,Resultado,Muestra,Edad,Granja
0,S-21-0055,?,2021-01-05,SIV,POSITIVO,Pulmón,NaN,NaN
1,S-21-0058,Huesca,2021-01-05,SIV,NEGATIVO,Lechón,Lechones abortados,NaN
2,S-21-0062,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN
3,S-21-0067,NaN,2021-01-05,SIV,NEGATIVO,Pulmón,NaN,NaN
4,S-21-0102,?,2021-01-08,SIV,NEGATIVO,Fluido oral,NaN,NaN
...,...,...,...,...,...,...,...,...
1039,S-23-9748,Huesca,2023-10-24,Lawsonia intracellularis,POSITIVO,Heces,12 semanas de vida,NaN
1040,S-23-9778,NaN,2023-10-25,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN
1041,S-23-9852,NaN,2023-10-26,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,NaN,NaN
1042,S-23-9921,NaN,2023-10-27,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,Transició,NaN


In [62]:
print(df['Patógeno'].unique())

['SIV' 'Lawsonia intracellularis' 'Mycoplasma hyopneumoniae']


In [63]:
print(df['Resultado'].unique())

['POSITIVO' 'NEGATIVO']


In [64]:
print(df['Muestra'].unique())

['Pulmón' 'Lechón' 'Fluido oral' 'Pulmón/ Fluido oral' 'Heces'
 'Raspado traqueal' 'Hisopo/ Raspado traqueal/ Pulmón' 'Intestino'
 'Hisopo' 'Raspado traqueal/ Pulmón' 'Fluido oral/ Pulmón'
 'Heces/intestino' 'Hisopo rectal' 'Porción intestinal' 'Exudado'
 'Pulmón ' 'Líquido pericárdico' 'Raspado traqueal/ Fluido oral'
 'Heces/hisopo rectal' 'Hisopo nasal' 'Hisopo nasal/ Pulmón/ Fluido Oral'
 'Órgano' 'Hisopo intestinal' 'Feto' 'Hisopo pulmonar' 'Heces/hisopos'
 'Ganglio' 'Intestino/heces/hisopo' 'Pulmón/ Fluido Oral'
 'Pulmón / Fluido Oral' 'Hisopo fecal' 'Cabeza']


In [65]:
print(df['Edad'].unique())

[nan 'Lechones abortados' '6-9 semanas' '3M0S' '40 kg' '50 Kg' 'Cerda'
 '2,5 meses' 'Recria' 's/n' 'Transició' 'Cebo' 'Destete'
 'Primerizas destetadas' 'Recría' 'Primerizas' 'Semanas 43 y 53'
 'Cerdas gestantes antes parto' 'Cerdo de cebo' '3 meses en cebo'
 'Final transición' '3 semanas en cebo' 'Fin de cebo-4 meses'
 '1 mes estancia en cebo' 'Cerdo 75 kg' '2 meses estancia'
 '14-18 semanas de vida' 'Final destete (6 semanas vida)'
 '24 semanas de vida' '16 semanas de vida' 'Multíparas/ primerizas'
 'Cerdas gestantes' 'Cerda de recría' '2 mesos' '10 setmanes' 'Cerdas'
 'Nulíparas/Primerizas destetadas' '10 semanas' '1 mes cebo'
 '3,5 meses cebo' 'Cerdas gestación' 'Cerdas nulíparas' '2 meses cebo'
 '2,5 meses cebo' '105 kg' 'Cerda lactante'
 '2 meses y medio estancia en cebo' 'Lechones 3-4 semanas postdestete'
 '3 meses de cebo' 'Primeriza' '11 semanas' '14 semanas' '18 semanas'
 '20 setmanes' '10 días de vida' 'Lechones 80 días' 'Lechones 60 días'
 '3 meses' 'Primerizas gestación' '

In [66]:
print(df['Granja'].unique())

[nan 'Madres reproductoras' 'Engreix' 'Engreix ' 'Engorde']


In [67]:
print(df['Provincia'].unique())

['?' 'Huesca' nan 'Zaragoza' 'Lleida' 'Treuel' 'Barcelona' 'Navarra'
 'Teruel' 'Tarragona' 'Huesca?' 'Málaga' 'Castellón' 'Almería' 'Ourense'
 'Soria' 'Teruel?' 'Girona' 'Múrcia?' 'Granada?' 'Múrcia' 'Albacete'
 'Castelló' 'Lleida?' 'La Rioja' 'Tarragona?']


In [68]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df['Age_cleaned'] = df['Edad'].apply(clean_text)
df['Farm_cleaned'] = df['Granja'].apply(clean_text)
df['Provincia'] = df['Provincia'].apply(clean_text)

print(df['Age_cleaned'].unique())

[None 'lechones abortados' '6-9 semanas' '3m0s' '40 kg' '50 kg' 'cerda'
 '2,5 meses' 'recria' 's/n' 'transicio' 'cebo' 'destete'
 'primerizas destetadas' 'primerizas' 'semanas 43 y 53'
 'cerdas gestantes antes parto' 'cerdo de cebo' '3 meses en cebo'
 'final transicion' '3 semanas en cebo' 'fin de cebo-4 meses'
 '1 mes estancia en cebo' 'cerdo 75 kg' '2 meses estancia'
 '14-18 semanas de vida' 'final destete (6 semanas vida)'
 '24 semanas de vida' '16 semanas de vida' 'multiparas/ primerizas'
 'cerdas gestantes' 'cerda de recria' '2 mesos' '10 setmanes' 'cerdas'
 'nuliparas/primerizas destetadas' '10 semanas' '1 mes cebo'
 '3,5 meses cebo' 'cerdas gestacion' 'cerdas nuliparas' '2 meses cebo'
 '2,5 meses cebo' '105 kg' 'cerda lactante'
 '2 meses y medio estancia en cebo' 'lechones 3-4 semanas postdestete'
 '3 meses de cebo' 'primeriza' '11 semanas' '14 semanas' '18 semanas'
 '20 setmanes' '10 dias de vida' 'lechones 80 dias' 'lechones 60 dias'
 '3 meses' 'primerizas gestacion' 'gestacio

In [69]:
print(df['Farm_cleaned'].unique())

[None 'madres reproductoras' 'engreix' 'engorde']


In [70]:
print(df['Provincia'].unique())

['?' 'huesca' None 'zaragoza' 'lleida' 'treuel' 'barcelona' 'navarra'
 'teruel' 'tarragona' 'huesca?' 'malaga' 'castellon' 'almeria' 'ourense'
 'soria' 'teruel?' 'girona' 'murcia?' 'granada?' 'murcia' 'albacete'
 'castello' 'lleida?' 'la rioja' 'tarragona?']


In [71]:
#Determine the PigStage by Age, Age Unit, FarmType. and map to one coulumn


df = df.rename(columns={
    'Id': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha ': 'Date',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Age_cleaned': 'Age', #Age of animal
    'Muestra': 'SampleType',
    'Farm_cleaned': 'FarmType', #age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "1"


# Combine 'Age' and 'Farm Type' columns into a new column 'Combined_Age'
df['Age'] = df['Age'].replace(0.0, None)
df['AnimalType'] = df[['Age', 'FarmType']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

# Replace to None if missing
df['AnimalType'] = df['AnimalType'].apply(lambda x: None if x == 'Missing' else x)

# Drop the original 'Edad' and 'Tipo Edad' columns if needed
df = df.drop(['Age', 'FarmType', 'Granja', 'Edad'], axis=1)



df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,AnimalType
0,S-21-0055,?,2021-01-05,SIV,POSITIVO,Pulmón,PCR,Spain,Meat,1,
1,S-21-0058,huesca,2021-01-05,SIV,NEGATIVO,Lechón,PCR,Spain,Meat,1,lechones abortados
2,S-21-0062,None,2021-01-05,SIV,NEGATIVO,Pulmón,PCR,Spain,Meat,1,
3,S-21-0067,None,2021-01-05,SIV,NEGATIVO,Pulmón,PCR,Spain,Meat,1,
4,S-21-0102,?,2021-01-08,SIV,NEGATIVO,Fluido oral,PCR,Spain,Meat,1,
...,...,...,...,...,...,...,...,...,...,...,...
1039,S-23-9748,huesca,2023-10-24,Lawsonia intracellularis,POSITIVO,Heces,PCR,Spain,Meat,1,12 semanas de vida
1040,S-23-9778,None,2023-10-25,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,PCR,Spain,Meat,1,
1041,S-23-9852,None,2023-10-26,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,PCR,Spain,Meat,1,
1042,S-23-9921,None,2023-10-27,Mycoplasma hyopneumoniae,NEGATIVO,Pulmón,PCR,Spain,Meat,1,transicio


In [72]:
print(df['AnimalType'].unique())

['' 'lechones abortados' '6-9 semanas' 'madres reproductoras' '3m0s'
 '40 kg' '50 kg' 'cerda' '2,5 meses' 'recria' 's/n' 'transicio'
 'cebo madres reproductoras' 'destete' 'primerizas destetadas'
 'recria madres reproductoras' 'primerizas' 'semanas 43 y 53'
 'cerdas gestantes antes parto' 'cerdo de cebo' '3 meses en cebo'
 'final transicion' '3 semanas en cebo' 'fin de cebo-4 meses'
 '1 mes estancia en cebo' 'cerdo 75 kg' '2 meses estancia'
 '14-18 semanas de vida' 'final destete (6 semanas vida)'
 '24 semanas de vida' '16 semanas de vida' 'multiparas/ primerizas'
 'cerdas gestantes' 'cerda de recria' 'primerizas madres reproductoras'
 'engreix' 'cebo' '2 mesos engreix' '10 setmanes engreix' 'cerdas'
 'nuliparas/primerizas destetadas'
 'nuliparas/primerizas destetadas madres reproductoras' '10 semanas'
 '1 mes cebo' '3,5 meses cebo' 'cerdas gestacion' 'cerdas nuliparas'
 '2 meses cebo' '2,5 meses cebo' '105 kg' 'cerda lactante'
 '2 meses y medio estancia en cebo' 'lechones 3-4 semanas 

In [73]:
#Annotations of different terminologies


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "POSITIVO": "1",
    "NEGATIVO": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")




# Add Pathogen mapping for annotation
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Mycoplasma hyopneumoniae": "Mycoplasma hyopneumoniae",
     "Lawsonia intracellularis": "Lawsonia intracellularis",
    
}
df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")



# SampleTypeMapping for sample types annotations
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Lechón': 'Tissue',
    'Fluido oral': 'Fluid',
    'Pulmón/ Fluido oral': 'Mixed',
    'Heces': 'Faeces',
    'Raspado traqueal': 'Swab',
    'Hisopo/ Raspado traqueal/ Pulmón': 'Mixed',
    'Intestino': 'Tissue',
    'Hisopo': 'Swab',
    'Raspado traqueal/ Pulmón': 'Mixed',
    'Fluido oral/ Pulmón': 'Mixed',
    'Heces/intestino': 'Mixed',
    'Hisopo rectal': 'Swab',
    'Porción intestinal': 'Tissue',
    'Exudado': 'Fluid',
    'Pulmón ': 'Tissue',
    'Líquido pericárdico': 'Fluid',
    'Raspado traqueal/ Fluido oral': 'Mixed',
    'Heces/hisopo rectal': 'Mixed',
    'Hisopo nasal': 'Swab',
    'Hisopo nasal/ Pulmón/ Fluido Oral': 'Mixed',
    'Órgano': 'Tissue',
    'Hisopo intestinal': 'Swab',
    'Feto': 'Tissue',
    'Hisopo pulmonar': 'Swab',
    'Heces/hisopos': 'Mixed',
    'Ganglio': 'Tissue',
    'Intestino/heces/hisopo': 'Mixed',
    'Pulmón/ Fluido Oral': 'Mixed',
    'Pulmón / Fluido Oral': 'Mixed',
    'Hisopo fecal': 'Swab',
    'Cabeza': 'Tissue',
}
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


print(df['SampleType'].unique())

['Tissue' 'Fluid' 'Mixed' 'Faeces' 'Swab']


In [74]:
# PigStages mapping
PigAgeMapping = {
    'lechones abortados': 'Sow',
    '6-9 semanas': 'Nursery',
    'madres reproductoras': 'Sow',
    '3m0s': 'Finishing',
    '40 kg': 'Finishing',
    '50 kg': 'Finishing',
    'cerda': 'Sow',
    '2,5 meses': 'Finishing',
    'recria': 'Gilt',
    's/n': 'Missing',
    'transicio': 'Nursery',
    'cebo madres reproductoras': 'Other',
    'destete': 'Weaning',
    'primerizas destetadas': 'Sow',
    'recria madres reproductoras': 'Mixed',
    'primerizas': 'Sow',
    'semanas 43 y 53': 'Other',
    'cerdas gestantes antes parto': 'Sow',
    'cerdo de cebo': 'Finishing',
    '3 meses en cebo': 'Finishing',
    'final transicion': 'Nursery',
    '3 semanas en cebo': 'Finishing',
    'fin de cebo-4 meses': 'Finishing',
    '1 mes estancia en cebo': 'Finishing',
    'cerdo 75 kg': 'Finishing',
    '2 meses estancia': 'Other',
    '14-18 semanas de vida': 'Finishing',
    'final destete (6 semanas vida)': 'Nursery',
    '24 semanas de vida': 'Finishing',
    '16 semanas de vida': 'Finishing',
    'multiparas/ primerizas': 'Sow',
    'cerdas gestantes': 'Sow',
    'cerda de recria': 'Gilt',
    'primerizas madres reproductoras': 'Sow',
    'engreix': 'Finishing',
    'cebo': 'Finishing',
    '2 mesos engreix': 'Finishing',
    '10 setmanes engreix': 'Finishing',
    'cerdas': 'Sow',
    'nuliparas/primerizas destetadas': 'Mixed',
    'nuliparas/primerizas destetadas madres reproductoras': 'Mixed',
    '10 semanas': 'Nursery',
    '1 mes cebo': 'Finishing',
    '3,5 meses cebo': 'Finishing',
    'cerdas gestacion': 'Sow',
    'cerdas nuliparas': 'Gilt',
    '2 meses cebo': 'Finishing',
    '2,5 meses cebo': 'Finishing',
    '105 kg': 'Finishing',
    'cerda lactante': 'Sow',
    '2 meses y medio estancia en cebo': 'Finishing',
    'lechones 3-4 semanas postdestete': 'Nursery',
    '3 meses de cebo': 'Finishing',
    'primeriza': 'Sow',
    '11 semanas': 'Finishing',
    '14 semanas': 'Finishing',
    '18 semanas': 'Finishing',
    '20 setmanes': 'Finishing',
    '10 dias de vida madres reproductoras': 'Suckling',
    'lechones 80 dias': 'Finishing',
    'lechones 60 dias': 'Nursery',
    '3 meses': 'Finishing',
    'primerizas gestacion': 'Sow',
    'gestacion': 'Sow',
    'transicion': 'Nursery',
    'gestants/ recria': 'Gilt',
    '1,5m': 'Nursery',
    '5 mesos': 'Finishing',
    'primeriza madres reproductoras': 'Sow',
    'truges': 'Sow',
    'cerdas adultas': 'Sow',
    'cerdas reposicion': 'Gilt',
    '1 mes estancia': 'Other',
    'truja 2 anys': 'Sow',
    'cerda recria': 'Gilt',
    '4 mesos': 'Finishing',
    '3 mesos': 'Finishing',
    'cerdo 25 semanas vida': 'Finishing',
    'lechon lactante': 'Suckling',
    '8-9 semanas': 'Nursery',
    '8-9 semanas vida': 'Nursery',
    '2m0s': 'Nursery',
    'lechon': 'Suckling',
    '8 semanas cebo': 'Finishing',
    'lechones 1 semana post-destete': 'Nursery',
    'lechon destetado': 'Nursery',
    'lechones/ recria': 'Mixed',
    '4 semanas': 'Weaning',
    '3 meses cebo': 'Finishing',
    '2 mesos': 'Finishing',
    'lechones': 'Suckling',
    'lechon 15 dias post destete': 'Nursery',
    'lechones 3-4 semanas': 'Weaning',
    'lechon transicion': 'Nursery',
    '1m3s': 'Nursery',
    '1,5 meses estancia': 'Other',
    'transicion madres reproductoras': 'Nursery',
    'llavora': 'Gilt',
    'lechones 2 semanas de vida': 'Suckling',
    '8 semanas': 'Nursery',
    '4 semanas de vida': 'Weaning',
    '12 setmanes': 'Finishing',
    'cerda primeriza': 'Sow',
    'lechones 2-3 semanas de vida': 'Suckling',
    '6 semanas': 'Nursery',
    '5 semanas': 'Nursery',
    '4, 5 meses': 'Finishing',
    'lechones 3 semanas post destete': 'Nursery',
    'lechones transicion': 'Nursery',
    '4-5 semanas de vida': 'Nursery',
    'cerdas primerizas madres reproductoras': 'Sow',
    'cerdas maternidad': 'Sow',
    '2-3 semanas de vida': 'Suckling',
    'lechones 16 kg': 'Nursery',
    'lechones 3 semanas': 'Weaning',
    'lechon 4 semanas post-destete': 'Nursery',
    '4,5 meses': 'Finishing',
    '20 dias cebo': 'Finishing',
    '10d engreix': 'Finishing',
    'cerdo 100 kg': 'Finishing',
    'reproductoras': 'Sow',
    '3,5 meses': 'Finishing',
    '6 setmanes de vida': 'Nursery',
    'truja': 'Sow',
    '2 meses engorde': 'Finishing',
    '2m3s': 'Finishing',
    'lechones 3,5 - 4 meses': 'Finishing',
    '5 meses': 'Finishing',
    'destete 2, 3 y 4 semanas, y cebo': 'Mixed',
    'lechones 4 semanas de vida': 'Weaning',
    '5 semanas vida; transicion': 'Nursery',
    'lechones post destete': 'Nursery',
    'lechones 12-14 semanas': 'Finishing',
    '10-12 semanas': 'Finishing',
    'lechones destete': 'Weaning',
    '6 semanas destetados': 'Nursery',
    '20 setmanes; recria': 'Gilt',
    '60 kg': 'Finishing',
    '30 dias': 'Nursery',
    'lechones 45 dias': 'Nursery',
    'cerdo 70 kg': 'Finishing',
    '55 kg': 'Finishing',
    '7 semanas de vida': 'Nursery',
    'cerdo 45 kg': 'Finishing',
    'lechones 4 semanas post destete': 'Nursery',
    'lechones 11-12 semanas de vida': 'Finishing',
    'cerdas 12 meses': 'Sow',
    '80 kg': 'Finishing',
    '21 dies': 'Suckling',
    'cerdos 30 kg': 'Finishing',
    'cerdo 110 kg': 'Finishing',
    'llavores': 'Gilt',
    'cerda gestante': 'Sow',
    '6 setmanes': 'Nursery',
    'lechones transicion madres reproductoras': 'Mixed',
    'destete transicion madres reproductoras': 'Mixed',
    '30-35 kg': 'Finishing',
    'lechones 7-8 semanas de vida': 'Nursery',
    '40-60 kg': 'Finishing',
    'lechones 30 kg': 'Finishing',
    'destete madres reproductoras': 'Weaning',
    'garrins destete 5-7 setmanes vida': 'Nursery',
    'precebo': 'Nursery',
    '50 dies': 'Nursery',
    '70 dies': 'Nursery',
    'lechones 3 semanas de vida': 'Weaning',
    '1 mes engorde': 'Finishing',
    '3-4 setmanes': 'Weaning',
    'cerdo 60 kg': 'Finishing',
    'lechones cebadero': 'Finishing',
    'lechones 5 y 25 dias de vida': 'Suckling',
    '90 dias': 'Finishing',
    '35 dias transicion': 'Nursery',
    '8 semanas vida': 'Nursery',
    'truja parideres': 'Sow',
    'cerdo 105 kg': 'Finishing',
    'lechones 10 dias post destete': 'Nursery',
    '2,5 mesos': 'Finishing',
    '0m1s': 'Suckling',
    '3 setmanes': 'Weaning',
    'nuliparas': 'Gilt',
    'cerda maternidad': 'Sow',
    '1 semana post-destete': 'Nursery',
    '9 semanas': 'Nursery',
    'lechon 9 semanas': 'Nursery',
    'madres': 'Sow',
    'lechon destete': 'Nursery',
    '>100d gestacion': 'Sow',
    'cerda recria madres reproductoras': 'Mixed',
    'cerdos en cebo': 'Finishing',
    'paridera/recria': 'Mixed',
    'parideras': 'Sow',
    'lechon paridera': 'Suckling',
    '6 mesos engorde': 'Finishing',
    'cerda 18 meses': 'Sow',
    'entrada cebo': 'Finishing',
    '4 setmanes': 'Weaning',
    '35 kg': 'Finishing',
    'cerda nulipara': 'Gilt',
    'lechones paridera': 'Suckling',
    'lechon destete madres reproductoras': 'Weaning',
    '5 setmanes': 'Nursery',
    '12 semanas de vida': 'Finishing',

}

df['PigStage'] = df['AnimalType'].map(PigAgeMapping).fillna('Missing')

print(df['PigStage'].unique())

['Missing' 'Sow' 'Nursery' 'Finishing' 'Gilt' 'Other' 'Weaning' 'Mixed'
 'Suckling']


In [75]:
# Province Mapping and annotations
# Province to Region (Autonomous Community) Mapping
ProvinceMapping = {
    '?': 'Missing',
    'albacete': 'Albacete',
    'almeria': 'Almeria',
    'barcelona': 'Barcelona',
    'castello': 'Castellon',
    'castellon': 'Castellon',
    'girona': 'Girona',
    'granada?': 'Missing',
    'huesca': 'Huesca',
    'huesca?': 'Missing',
    'la rioja': 'LaRioja',
    'lleida': 'Lleida',
    'lleida?': 'Missing',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'murcia?': 'Missing',
    'navarra': 'Navarre',
    'None': 'Missing',
    'ourense': 'Ourense',
    'soria': 'Soria',
    'tarragona': 'Tarragona',
    'tarragona?': 'Missing',
    'teruel': 'Teruel',
    'teruel?': 'Missing',
    'treuel': 'Teruel',
    'zaragoza': 'Zaragoza',

}


df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")

# Convert 'Date' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
# Format 'Date' column as 'DD-MM-YYYY'
#df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')

# Drop intermediate columns if needed
df.drop(['AnimalType'], axis=1, inplace=True)

In [76]:
df.head (50)

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,S-21-0055,Missing,2021-01-05,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Missing
1,S-21-0058,Huesca,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Sow
2,S-21-0062,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
3,S-21-0067,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
4,S-21-0102,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
5,S-21-0106,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Nursery
6,S-21-0156,Huesca,2021-01-12,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Sow
7,S-21-0223,Missing,2021-01-14,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
8,S-21-0270,Missing,2021-01-15,Swine influenza virus,0,Mixed,PCR,Spain,Meat,1,Missing
9,S-21-0274,Missing,2021-01-15,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing


In [77]:
print(df['Pathogen'].unique())

['Swine influenza virus' 'Lawsonia intracellularis'
 'Mycoplasma hyopneumoniae']


In [78]:
print(df['Result'].unique())

['1' '0']


In [79]:
print(df['Province'].unique())

['Missing' 'Huesca' 'Zaragoza' 'Lleida' 'Teruel' 'Barcelona' 'Navarre'
 'Tarragona' 'Malaga' 'Castellon' 'Almeria' 'Ourense' 'Soria' 'Girona'
 'Murcia' 'Albacete' 'LaRioja']


In [80]:
#df = df.drop(['Age'], axis=1)
#df = df.drop(['Granja'], axis=1)
#df = df.drop(['FarmType'], axis=1)

In [81]:
df.head (50)


,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,S-21-0055,Missing,2021-01-05,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Missing
1,S-21-0058,Huesca,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Sow
2,S-21-0062,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
3,S-21-0067,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing
4,S-21-0102,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
5,S-21-0106,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Nursery
6,S-21-0156,Huesca,2021-01-12,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Sow
7,S-21-0223,Missing,2021-01-14,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing
8,S-21-0270,Missing,2021-01-15,Swine influenza virus,0,Mixed,PCR,Spain,Meat,1,Missing
9,S-21-0274,Missing,2021-01-15,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing


In [82]:
df.to_csv('output/PigLab1_1.csv', index=False)

## Second data upload

In [83]:
# Define the file paths
files1 = ['Lab1/Biofar Laboratoris_DECIDE_2024.xlsx']

# Load the data into Pandas dataframes
dfs1 = []
for file in files1:
    df1 = pd.read_excel(file, engine='openpyxl')
    dfs1.append(df1)

df1 = dfs1[0]
df1

,Nº,Muestras enviadas,Descripción.1,Fecha toma de muestra,Província,Tipo de explotación,Edad,Valor
0,1,ADN - ADN,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,NaN,NaN,NaN,POSITIVO
1,2,ADN - ADN,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,NaN,NaN,NaN,POSITIVO
2,3,ADN - ADN,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,NaN,NaN,NaN,POSITIVO
3,4,ADN - ADN,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,NaN,NaN,NaN,POSITIVO
4,5,ADN - ADN,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,NaN,NaN,NaN,POSITIVO
...,...,...,...,...,...,...,...,...
13726,13727,Sangre,PCR PRRSV NA,2024-01-09,Huesca,Producción de lechones,NaN,NEGATIVO
13727,13728,Sangre,PCR PRRSV EU,2024-01-09,Huesca,Centros de recogida de semen,NaN,NEGATIVO
13728,13729,Sangre,PCR PRRSV NA,2024-01-09,Huesca,Centros de recogida de semen,NaN,NEGATIVO
13729,13730,Sangre,PCR PRRSV EU,2024-01-09,Lérida,Centros de recogida de semen,NaN,NEGATIVO


In [84]:
#print(df1['Descripción.1'].unique())

In [85]:
#print(df1['Muestras enviadas'].unique())

In [86]:
#print(df1['Província'].unique())

In [87]:
#print(df1['Tipo de explotación'].unique())

In [88]:
#print(df1['Edad'].unique())

In [89]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df1['Age_cleaned'] = df1['Edad'].apply(clean_text)
df1['Farm_cleaned'] = df1['Tipo de explotación'].apply(clean_text)
df1['Provincia'] = df1['Província'].apply(clean_text)
df1['Muestras'] = df1['Muestras enviadas'].apply(clean_text)

#print(df1['Age_cleaned'].unique())

In [90]:
#print(df1['Farm_cleaned'].unique())

In [91]:
print(df1['Provincia'].unique())

[None 'tarragona' 'huesca' 'zaragoza' 'barcelona' 'lerida' 'gerona'
 'granada' 'valencia' 'sevilla' 'almeria' 'murcia' 'badajoz' 'soria'
 'segovia' 'navarra' 'castellon' 'teruel' 'la rioja' 'cuenca' 'toledo' '-'
 'caceres' 'zamora' 'orense' 'cordoba' 'albacete' 'malaga' 'salamanca'
 'valladolid' 'las palmas' '0']


In [92]:
df1['Muestras'] = df1['Muestras'].str.replace(" -", '', regex=False).str.strip()
#print(df1['Muestras'].unique())

In [93]:
#Determine the PigStage by Age, Age Unit, FarmType. and map to one coulumn


df1 = df1.rename(columns={
    'Nº': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha toma de muestra': 'Date',
    'Descripción.1': 'Pathogen',
    'Valor': 'Result',
    'Age_cleaned': 'Age', #Age of animal
    'Muestras': 'SampleType',
    'Farm_cleaned': 'FarmType', #age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df1['DiagnosticTest'] = 'PCR'
df1['Country'] = 'Spain'
df1['Breed'] = 'Meat'
df1["Lab_Reference"] = "1"


# Combine 'Age' and 'Farm Type' columns into a new column 'Combined_Age'
df1['Age'] = df1['Age'].replace(0.0, None)
df1['AnimalType'] = df1[['Age', 'FarmType']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

# Replace to None if missing
df1['AnimalType'] = df1['AnimalType'].apply(lambda x: None if x == 'Missing' else x)

# Drop the original 'Edad' and 'Tipo Edad' columns if needed
df1 = df1.drop(['Age', 'FarmType', 'Muestras enviadas', 'Província', 'Tipo de explotación', 'Edad'], axis=1)




df1

,SampleNumber,Pathogen,Date,Result,Province,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,AnimalType
0,1,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,POSITIVO,None,adn adn,PCR,Spain,Meat,1,
1,2,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,POSITIVO,None,adn adn,PCR,Spain,Meat,1,
2,3,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,POSITIVO,None,adn adn,PCR,Spain,Meat,1,
3,4,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,POSITIVO,None,adn adn,PCR,Spain,Meat,1,
4,5,PCR serotipificación Actinobacillus pleuropneu...,2024-12-31,POSITIVO,None,adn adn,PCR,Spain,Meat,1,
...,...,...,...,...,...,...,...,...,...,...,...
13726,13727,PCR PRRSV NA,2024-01-09,NEGATIVO,huesca,sangre,PCR,Spain,Meat,1,produccion de lechones
13727,13728,PCR PRRSV EU,2024-01-09,NEGATIVO,huesca,sangre,PCR,Spain,Meat,1,centros de recogida de semen
13728,13729,PCR PRRSV NA,2024-01-09,NEGATIVO,huesca,sangre,PCR,Spain,Meat,1,centros de recogida de semen
13729,13730,PCR PRRSV EU,2024-01-09,NEGATIVO,lerida,sangre,PCR,Spain,Meat,1,centros de recogida de semen


In [94]:
df1['Result'] = df1['Result'].apply(clean_text)
print(df1['Result'].unique())

['positivo' 'negativo' 'no concluyente']


In [95]:
#Annotations of different terminologies


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "positivo": "1",
    "negativo": "0",
    "no concluyente": "Missing",
}
df1["Result"] = df1["Result"].map(resultMapping).fillna("Missing")




# Add Pathogen mapping for annotation
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Mycoplasma hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Lawsonia intracellularis": "Lawsonia intracellularis",
    'PCR serotipificación Actinobacillus pleuropneumoniae': 'Actinobacillus pleuropneumoniae',
    'PCR PRRSV EU': 'Porcine reproductive and respiratory syndrome virus',
    'PCR PRRSV NA': 'Porcine reproductive and respiratory syndrome virus',
    'PCR PEDV': 'Porcine epidemic diarrhea virus',
    'PCR Influenza A': 'Swine influenza virus',
    'PCR Actinobacillus pleuropneumoniae': 'Actinobacillus pleuropneumoniae',
    'PCR Brachyspira hyodysenteriae': 'Brachyspira hyodysenteriae',
    'Cuantificación PCR PRRSV EU': 'Porcine reproductive and respiratory syndrome virus',
    'PCR Mycoplasma hyopneumoniae': 'Mycoplasma hyopneumoniae',
    'PCR Lawsonia': 'Lawsonia intracellularis',
    'Brachyspira hyodysenteriae': 'Brachyspira hyodysenteriae',
    'Brachyspira pilosicoli': 'Brachyspira spp',
    'PCR serotipificación Actinobacillus pleuropneumoniae en ADN': 'Actinobacillus pleuropneumoniae',

    
}
df1["Pathogen"] = df1["Pathogen"].map(PathogenMapping).fillna("Missing")



# SampleTypeMapping for sample types annotations
SampleTypeMapping = {
    'adn  adn': 'Other',
    'sangre  , fluido oral': 'Mixed',
    'sangre': 'Blood',
    'hisopo fecal': 'Swab',
    'fluido oral': 'Fluid',
    'pulmon': 'Tissue',
    'sangre  , hisopo fecal': 'Mixed',
    'lengua': 'Tissue',
    'hez': 'Faeces',
    'hisopo sangre': 'Mixed',
    'semen  , sangre': 'Mixed',
    'semen': 'Fluid',
    'sangre  , pulmon': 'Mixed',
    'hisopo sangre  , sangre': 'Mixed',
    'pulmon  , higado': 'Mixed',
    'hisopo': 'Swab',
    'sangre  , lengua': 'Mixed',
    'macerado de lenguas, macerado de colas': 'Mixed',
    'cola  , hisopo rectal': 'Mixed',
    'tejidos y organos': 'Mixed',
    'sangre  , hisopo rectal': 'Mixed',
    'hisopo sangre  , semen': 'Mixed',
    'pulmon  , liquido pericardico  , liquido abdominal': 'Mixed',
    'fluido testicular': 'Fluid',
    'macerado de lenguas  macerado de lenguas, macerado de colas': 'Mixed',
    'hisopo nasal': 'Swab',
    'fluido oral  , pulmon  , liquido pericardico': 'Mixed',
    'lengua  , cola': 'Mixed',
    'sangre  , cabeza': 'Mixed',
    'hez  , extremidad': 'Mixed',
    'intestino': 'Tissue',
    'hisopo rectal  , intestino': 'Mixed',
    'fluido oral  , liquido pericardico': 'Mixed',
    'sangre  , lengua  , feto  feto': 'Mixed',
    'pulmon  , corazon  corazon': 'Mixed',
    'sangre  , hisopo sangre  , semen': 'Mixed',
    'sangre  , semen': 'Mixed',
    'suero': 'Blood',
    'macerado de colas': 'Fluid',
    'sangre  , hez': 'Mixed',
    'fluido oral  , pulmon': 'Mixed',
    'feto  feto': 'Tissue',
    'sangre  , feto  feto': 'Mixed',
    'fluido oral  , lengua': 'Mixed',
    'lengua  , pulmon': 'Mixed',
    'lechon': 'Tissue',
    'hisopo rectal': 'Swab',
    'pulmon  , ganglio  ganglio, intestino': 'Mixed',
    'macerado de colas  , macerado de lenguas  macerado de lenguas': 'Mixed',
    'hez  , intestino': 'Mixed',
    'sangre  , hisopo sangre': 'Mixed',
    'sangre  , pulmon  , tonsila  tonsila': 'Mixed',
    'sangre  , macerado de colas  , macerado de lenguas  macerado de lenguas': 'Mixed',
    'intestino  , hez': 'Mixed',
    'sangre  , hisopo nasal': 'Mixed',
    'sangre  , lechon': 'Mixed',
    'pulmon y corazon': 'Mixed',
    'lengua  , hez': 'Mixed',
    'sangre  , arn  arn, vacuna  vacuna, nfw  nfw': 'Mixed',
    'fluido oral  , cabeza': 'Mixed',
    'hez  , adn  adn': 'Mixed',
    'fluido de procesado': 'Fluid',
    'dosis seminal': 'Fluid',
    'sangre , hisopo sangre': 'Mixed',
    'arn  arn': 'Other',
    'pulmon  , higado  , bazo': 'Mixed',
    'sangre  , macerado de lenguas  macerado de lenguas, macerado de colas': 'Mixed',
    'pulmon  , hisopo pericardico  , tejidos y organos': 'Mixed',
    'aborto  aborto': 'Tissue',
    'cola  , hez': 'Mixed',
    'hisopo traqueobronquial': 'Swab',
    'hez  , lechon': 'Mixed',
    'sangre  , pulmon  , liquido pericardico': 'Mixed',
    'sangre  , hez  , hisopo fecal': 'Mixed',
    'macerado de lenguas  macerado de lenguas, macerado de colas , fluido oral': 'Mixed',
    'intestino  , hisopo intestinal': 'Mixed',
    'hisopo rectal  , hez': 'Mixed',
    'cultivo  cultivo': 'Other',
    'sangre  , intestino': 'Mixed',
    'lengua , macerado de colas': 'Mixed',
    'pulmon  , lechon  , intestino': 'Mixed',
    'sangre  , lengua  , intestino  , aborto  aborto': 'Mixed',
    'pulmon y tonsila': 'Mixed',
    'sangre  , fluido de procesado': 'Mixed',
    'fluido de procesado  , fluido oral': 'Mixed',
    'hez  , contenido intestinal': 'Mixed',
    'pulmon  , extremidad  , cabeza': 'Mixed',
    'hisopo fibrina , bazo': 'Mixed',
    'hez  , hisopo rectal': 'Mixed',
    'pulmon , bazo': 'Mixed',
    'placenta': 'Tissue',
    'contenido intestinal': 'Faeces',
    'sangre  , hisopo rectal  , lechon': 'Mixed',
    'pulmon  , intestino': 'Mixed',
    'porcion intestinal': 'Tissue',
    'pulmon  , higado  , bazo  , ganglio  ganglio': 'Mixed',
    'hisopo sangre  , vacuna  vacuna, agua  , arn  arn': 'Mixed',
    'sangre  , cola  , lengua': 'Mixed',
    'lengua  , agua': 'Mixed',
    'sangre  , macerado de lenguas  macerado de lenguas, macerado de colas  , fluido oral': 'Mixed',
    'sangre  , fluido oral  , agua': 'Mixed',
    'sangre  , agua': 'Mixed',
    'sangre  , fluido oral  , lengua  , feto  feto': 'Mixed',
    'fluido oral  , hisopo pericardico  , liquido abdominal': 'Mixed',
    'pulmon  , ganglio  ganglio': 'Mixed',
    'sangre  , fluido oral  , macerado de colas  , macerado de lenguas  macerado de lenguas': 'Mixed',
    'lengua  , cola  , testiculo  testiculo': 'Mixed',
    'lengua  , testiculo  testiculo': 'Mixed',
    'testiculo  testiculo, lengua': 'Mixed',
    'pulmon  , liquido  liquido': 'Mixed',
    'fluido oral, lengua , cola': 'Mixed',
    'liquido toracico': 'Fluid',
    'liquido pericardico  , liquido abdominal': 'Mixed',
    'pulmon  , intestino  , bazo  , higado  , rinon': 'Mixed',
    'hisopo oral': 'Swab',
    'hisopo sangre, sangre': 'Mixed',
    'cola  , lengua': 'Mixed',
    'pulmon  , liquido pericardico  , extremidad': 'Mixed',
    'hisopo cutaneo  , lechon': 'Mixed',
    'semen  , hisopo sangre': 'Mixed',
    'pulmon  , tonsila  tonsila': 'Mixed',
    'cola  , testiculo  testiculo, lengua': 'Mixed',
    'testiculo  testiculo': 'Tissue',
    'macerado de lenguas  macerado de lenguas, fluido oral  , agua': 'Mixed',
    'sangre  , fluido oral  , hez  , intestino': 'Mixed',
    'sangre  , cordon umbilical  , lengua': 'Mixed',
    'hez  , agua': 'Mixed',
    'lengua  , cabeza': 'Mixed',
    'arn': 'Other',
    'cola  , testiculo  testiculo': 'Mixed',
    'sangre  , testiculo  testiculo': 'Mixed',
    'macerado de colas  , macerado de lenguas  macerado de lenguas, fluido oral': 'Mixed',
    'macerado de colas  , lengua  , hez': 'Mixed',
    'fluido testicular  , testiculo  testiculo': 'Mixed',
    'sangre , hisopo rectal': 'Mixed',
    'fibrina  , bazo': 'Mixed',
    'macerado de lenguas  macerado de lenguas': 'Mixed',
    'lengua  , feto  feto': 'Mixed',
    'fluido oral  , pulmon  , hisopo pericardico  , hisopo articular': 'Mixed',
    'hisopo rectal  , hisopo vaginal': 'Mixed',
    'fluido oral  , hisopo': 'Mixed',
    'sangre, lengua': 'Mixed',
    'sangre  , fluido oral  , organo  organo': 'Mixed',
    'fluido oral  , exudado': 'Mixed',
    'sangre  , macerado de lenguas  macerado de lenguas': 'Mixed',
    'semen, hisopo sangre': 'Mixed',
    'fluido oral  , pulmon  , intestino': 'Mixed',
    'sangre , arn': 'Mixed',
    'hisopo fecal  , intestino': 'Mixed',
    'fluido oral  , liquido abdominal': 'Mixed',
    'pulmon  , liquido pericardico': 'Mixed',
    'sangre  , lengua  , aborto  aborto': 'Mixed',
    'sangre  , aborto  aborto': 'Mixed',
    'sangre  , calzas': 'Mixed',
    'pulmon, cabeza, liquido pulmonar, liquido articular': 'Mixed',
    'sangre , lengua , intestino': 'Mixed',
    'cabeza  , pulmon  , liquido pulmonar  , hez': 'Mixed',
    'cabeza  , hez': 'Mixed',
    'cabeza  , pulmon': 'Mixed',
    'hez  , sangre': 'Mixed',
    'sangre  , raspado traqueal': 'Mixed',
    'lengua  , fluido oral': 'Mixed',
    'hisopo sangre  , vacuna  vacuna, nfw  nfw, arn  arn': 'Mixed',
    'hisopo sangre, semen': 'Mixed',
    'sangre  , lengua  , cola': 'Mixed',
    'cola  , lengua  , testiculo  testiculo': 'Mixed',
    'macerado de colas  macerado de colas, macerado de lenguas  macerado de lenguas': 'Mixed',
    'macerado de lenguas': 'Fluid',
    'pulmon  , hisopo pericardico': 'Mixed',
    'lengua , testiculo': 'Mixed',
    'cola, testiculo, lengua': 'Mixed',
    'cola': 'Tissue',
    'fluido oral  fluido oral': 'Fluid',
    'semen  semen': 'Fluid',
    'feto': 'Tissue',
    'hisopo de sangre': 'Swab',
    'hisopo intestinal': 'Swab',
    'adn': 'Other',
    'intestinos': 'Tissue',
    'lenguas': 'Tissue',
    'rna': 'Other',
    'testiculos': 'Tissue',
    'dosis de semen': 'Fluid',
    'lechones': 'Tissue',
    'raspado': 'Swab',
    'colas': 'Tissue',
    'colas y testiculos': 'Mixed',
    'hisopos de sangre': 'Swab',
    'pulmlon': 'Tissue',
    'sangres': 'Blood',
    'hisopos sangre': 'Mixed',
    'dosis semen': 'Fluid',
    'fluido procesado': 'Fluid',

}
df1["SampleType"] = df1["SampleType"].map(SampleTypeMapping).fillna("Missing")



print(df1['SampleType'].unique())

['Other' 'Mixed' 'Blood' 'Swab' 'Fluid' 'Tissue' 'Faeces']


In [96]:
print(df1['Pathogen'].unique())

['Actinobacillus pleuropneumoniae'
 'Porcine reproductive and respiratory syndrome virus'
 'Porcine epidemic diarrhea virus' 'Swine influenza virus'
 'Brachyspira hyodysenteriae' 'Mycoplasma hyopneumoniae'
 'Lawsonia intracellularis' 'Brachyspira spp']


In [97]:
#print(df1['AnimalType'].unique())

In [98]:
# PigStages mapping
PigStageMapping = {
  '': 'Missing',
'lechones 28 dias post-destete / cerdas reposicion produccion de lechones': 'Mixed',
'lechones 14 dias post-destete produccion de lechones': 'Nursery',
'verracos centros de recogida de semen': 'Boar',
'centros de recogida de semen': 'Boar',
'cebo o cebadero': 'Finishing',
'seleccion': 'Gilt',
'multiplicacion': 'Missing',
'* produccion de lechones': 'Missing',
'produccion de lechones': 'Missing',
'22 dias produccion de lechones': 'Weaning',
'llavora produccion de lechones': 'Gilt',
'sortida recria de reproductores': 'Gilt',
'3 meses y medio cebo o cebadero': 'Finishing',
'transicion transicion de lechones': 'Nursery',
'lechones produccion de lechones': 'Suckling',
'verracos adultos centros de recogida de semen': 'Boar',
'150 dias recria de reproductores': 'Gilt',
'lechones tipo mixto': 'Suckling',
'3 meses de vida tipo mixto': 'Finishing',
'recria de reproductores': 'Gilt',
'cedas produccion de lechones': 'Sow',
'recria produccion de lechones': 'Gilt',
'adultos centros de recogida de semen': 'Boar',
'6 meses centros de recogida de semen': 'Boar',
'25-30 kg cebo o cebadero': 'Finishing',
'4 meses cebo o cebadero': 'Finishing',
'lechones 26 dias pre destete tipo mixto': 'Nursery',
'tipo mixto': 'Other',
'lechones +7 semanas de vida tipo mixto': 'Nursery',
'50 kg recria de reproductores': 'Gilt',
'30 semanas produccion de lechones': 'Finishing',
'lechones paridera produccion de lechones': 'Suckling',
'28 dies engreix cebo o cebadero': 'Finishing',
'3 meses': 'Finishing',
'3 meses recria de reproductores': 'Gilt',
'sangs gestants / pulmo transicio produccion de lechones': 'Mixed',
'truja produccion de lechones': 'Sow',
'4 meses transicion de lechones': 'Finishing',
'truges produccion de lechones': 'Sow',
'27 dias produccion de lechones': 'Weaning',
'lechones semanas post destete produccion de lechones': 'Nursery',
'adulto centros de recogida de semen': 'Boar',
'25 semanas produccion de lechones': 'Finishing',
'6 meses sin datos': 'Finishing',
'23 dias produccion de lechones': 'Weaning',
'<28 dias produccion de ciclo cerrado': 'Nursery',
'indistinta cebo o cebadero': 'Finishing',
'lechones 5-9 semanas de vida transicion de lechones': 'Nursery',
'60 dias cebo o cebadero': 'Finishing',
'50 kg cebo o cebadero': 'Finishing',
'7 semanas transicion de lechones': 'Nursery',
'lechon paridera tipo mixto': 'Suckling',
'4 semanas produccion de lechones': 'Weaning',
'10 semanas recria de reproductores': 'Gilt',
'transicion de lechones': 'Nursery',
'3 semanas produccion de lechones': 'Weaning',
'futuras reproductoras 8-18 semanas produccion de lechones': 'Gilt',
'7 semanas cebo o cebadero': 'Nursery',
'lechon 3 semanas produccion de lechones': 'Weaning',
'lechon 3 semanas': 'Weaning',
'lechones 3 dias (hisopos) / lechones 10 dias (intestino) produccion de lechones': 'Suckling',
'4 meses multiplicacion': 'Gilt',
'21 dias produccion de lechones': 'Weaning',
'42 dias de vida produccion de lechones': 'Nursery',
'16 semanas cebo o cebadero': 'Finishing',
'40kg cebo o cebadero': 'Finishing',
'3 mesos': 'Finishing',
'40-50kg': 'Finishing',
'dia centros de recogida de semen': 'Boar',
'7 meses produccion de lechones': 'Finishing',
'24 semanas produccion de lechones': 'Finishing',
'25 dias produccion de lechones': 'Weaning',
'7 semanas produccion de lechones': 'Nursery',
'lechones 2 semanas produccion de lechones': 'Suckling',
'11 semanas transicion de lechones': 'Finishing',
'lechones nacidos muertos produccion de lechones': 'Sow',
'lnm 3-4 semanas produccion de lechones': 'Weaning',
'1a entrada 7 meses - 2a entrada 5,5 meses recria de reproductores': 'Mixed',
'9-25 semanas multiplicacion': 'Mixed',
'sangre 21 dias / leguas 0 dias produccion de lechones': 'Mixed',
'lechon transicion transicion de lechones': 'Nursery',
'6 meses recria de reproductores': 'Gilt',
'2 meses recria de reproductores': 'Gilt',
'20 dias produccion de lechones': 'Suckling',
'3 dias produccion de lechones': 'Suckling',
'produccion de ciclo cerrado': 'Other',
'* tipo mixto': 'Missing',
'lechon 8 semanas de edad transicion de lechones': 'Nursery',
'5 semanas tipo mixto': 'Nursery',
'6 kg cebo o cebadero': 'Nursery',
'15 semanas produccion de lechones': 'Finishing',
'*': 'Missing',
'cerdo 80kg': 'Finishing',
'3 y 6 semanas produccion de lechones': 'Mixed',
'2 semanas post destete cebo o cebadero': 'Nursery',
'4-5/7/10 semanas produccion de lechones': 'Nursery',
'18 semanas cebo o cebadero': 'Finishing',
'6 semanas de vida cebo o cebadero': 'Nursery',
'transicio transicion de lechones': 'Nursery',
'9 semanas multiplicacion': 'Nursery',
'trujes produccion de lechones': 'Sow',
'28 dias tipo mixto': 'Weaning',
'nacidos muertos produccion de lechones': 'Sow',
'lechon produccion de lechones': 'Suckling',
'lechones pre destete produccion de lechones': 'Suckling',
'0 produccion de lechones': 'Missing',
'35/28/21 dias post destete produccion de lechones': 'Nursery',
'3-4 dias produccion de lechones': 'Suckling',
'45-50 kg cebo o cebadero': 'Finishing',
'7 semanas de vida transicion de lechones': 'Nursery',
'25-28 dias predestete tipo mixto': 'Suckling',
'lechones nacidos muertos tipo mixto': 'Sow',
'2 semanas entrada en cebo cebo o cebadero': 'Finishing',
'35 kg cebo o cebadero': 'Finishing',
'reposicion produccion de lechones': 'Gilt',
'reposicion recria de reproductores': 'Gilt',
'3 - 4,5 semanas multiplicacion': 'Weaning',
'4-8-12 semanas multiplicacion': 'Mixed',
'transicion 6a semana transicion de lechones': 'Nursery',
'transicion 1a semana transicion de lechones': 'Nursery',
'reposicion seleccion': 'Gilt',
'garrins 4 dies produccion de lechones': 'Suckling',
'1-3 dias produccion de lechones': 'Suckling',
'garrins transicio': 'Nursery',
'3 semanas -': 'Weaning',
'sortida multiplicacion': 'Gilt',
'lechones +2 semanas destetados produccion de lechones': 'Mixed',
'transicion 5a semana transicion de lechones': 'Nursery',
'cerda produccion de lechones': 'Sow',
'32 dies post-desmam transicion de lechones': 'Nursery',
'34 semanas multiplicacion': 'Gilt',
'lechones 7 semanas': 'Nursery',
'semana 45 produccion de lechones': 'Finishing',
'1-2 dias de vida seleccion': 'Suckling',
'6 mesos cebo o cebadero': 'Finishing',
'4 semanas': 'Weaning',
'cerdas produccion de lechones': 'Sow',
'10-12 dias de vida produccion de ciclo cerrado': 'Suckling',
'35 kg recria de reproductores': 'Finishing',
'garrins 1 dia': 'Suckling',
'9 semanas produccion de lechones': 'Nursery',
'llavores sortida multiplicacion': 'Gilt',
'1 mes cebo o cebadero': 'Finishing',
'2-3 dias tipo mixto': 'Suckling',
'50 kg multiplicacion': 'Gilt',
'18 kg cebo o cebadero': 'Nursery',
'cerdo reposicion tipo mixto': 'Gilt',
'mascles': 'Boar',
'14-16 semanas cebo o cebadero': 'Finishing',
'lechones 26 dias predestete tipo mixto': 'Weaning',
'15-21 dias multiplicacion': 'Suckling',
'3 semanas de vida produccion de lechones': 'Weaning',
'9 meses': 'Finishing',
'24 dias produccion de lechones': 'Weaning',
'1,5 meses transicion de lechones': 'Nursery',
'2 meses cebo o cebadero': 'Finishing',
'cerdas en produccion multiplicacion': 'Sow',
'<3 / 13 semanas de vida tipo mixto': 'Mixed',
' "15-21 dies dentrada a lengreix cebo o cebadero"': 'Finishing',
'7 meses produccion de ciclo cerrado': 'Finishing',
'transicion': 'Nursery',
'21-30 dies post-desmam transicion de lechones': 'Nursery',
'5 meses recria de reproductores': 'Gilt',
'5 meses cebo o cebadero': 'Finishing',
'20 - 23 - 26 semanas multiplicacion': 'Gilt',
'1 mes': 'Weaning',
'lechones maternidad 5 dias produccion de lechones': 'Suckling',
'45 a 47 semanas': 'Finishing',
'15 dias de vida produccion de lechones': 'Suckling',
'lechones 5 semanas post destete produccion de lechones': 'Nursery',
'6 meses': 'Finishing',
'truges multiplicacion': 'Sow',
'10-15 semanas multiplicacion': 'Gilt',
'3,5 meses cebo o cebadero': 'Finishing',
'10 semanas cebo o cebadero': 'Finishing',
'6 meses cebo o cebadero': 'Finishing',
'lechon 20 dias lact produccion de lechones': 'Suckling',
'lechones destete produccion de lechones': 'Weaning',
'multiparas y primerizas produccion de lechones': 'Sow',
'4-6 dias produccion de lechones': 'Suckling',
'final de cebo produccion de lechones': 'Finishing',
'reproductoras': 'Sow',
'destete cebo o cebadero': 'Weaning',
'lechones 2a semana produccion de lechones': 'Suckling',
'6/8 semanas tipo mixto': 'Nursery',
'2,5 meses cebo o cebadero': 'Finishing',
'10 dias de vida produccion de lechones': 'Suckling',
'lechones paridera 15 dias vida produccion de lechones': 'Suckling',
'2 dias produccion de lechones': 'Suckling',
'3-4 semanas transicion de lechones': 'Weaning',
'11 semanas produccion de lechones': 'Finishing',
'cerdo final cebo produccion de lechones': 'Finishing',
'cerdas reproductoras produccion de lechones': 'Sow',
'lechones parto s.45 tipo mixto': 'Suckling',
'lechon destetado produccion de lechones': 'Weaning',
'lechones 5 semanas transicion de lechones': 'Nursery',
'lechones* produccion de lechones': 'Suckling',
'10 dias produccion de lechones': 'Suckling',
'1 semana produccion de lechones': 'Suckling',
'sin datos': 'Missing',
'entrada produccion de lechones': 'Nursery',
'12 dias y 11 semanas multiplicacion': 'Mixed',
'2 meses cebo cebo o cebadero': 'Finishing',
'3 mesos engreix cebo o cebadero': 'Finishing',
'lechones': 'Suckling',
'4 dias multiplicacion': 'Suckling',
'lechon transicion produccion de lechones': 'Nursery',
'5 semanas transicion de lechones': 'Nursery',
'lechones mortinatos produccion de lechones': 'Sow',
'lechones mortinatos, dia produccion de lechones': 'Sow',
'semana 1-2': 'Suckling',
'cerdas gestantes produccion de lechones': 'Sow',
'cebo cebo o cebadero': 'Finishing',
'cebo -': 'Finishing',
'llavores produccion de lechones': 'Gilt',
'1 mes i 1/2 cebo o cebadero': 'Finishing',
'2 dies de vida produccion de lechones': 'Suckling',
'6 semanas transicion de lechones': 'Nursery',
'2 dies post destete tipo mixto': 'Nursery',
'2 dies post destete transicion de lechones': 'Nursery',
'transicion produccion de lechones': 'Nursery',
'destete transicion de lechones': 'Weaning',
'truja 1r part produccion de lechones': 'Sow',
'llavores seleccion': 'Gilt',
'22 dies de vida produccion de lechones': 'Suckling',
'45kg cebo o cebadero': 'Finishing',
'4-5 semanas cebo o cebadero': 'Finishing',
'* multiplicacion': 'Missing',
'-': 'Missing',
'5 y 7 meses recria de reproductores': 'Gilt',
'1 dia, 2 sem post destete -': 'Mixed',
'9 semanas transicion de lechones': 'Nursery',
'sangre: 21 dias / lenguas: 0 dias produccion de lechones': 'Suckling',
'10 semanas produccion de lechones': 'Nursery',
'semana 45 transicion de lechones': 'Nursery',
'budell: destete / diarrea: truges': 'Mixed',
'pre-destete tipo mixto': 'Suckling',
'nm produccion de lechones': 'Missing',
'lechones 10 dias produccion de lechones': 'Suckling',
'destete produccion de lechones': 'Weaning',
'5 setmanes transicion de lechones': 'Nursery',
'destete 1, 2, 3 i 4 setmanes produccion de lechones': 'Nursery',
'3 meses cebo o cebadero': 'Finishing',
'2-3 dias produccion de lechones': 'Suckling',
'35-40 dias tipo mixto': 'Nursery',
'machos de cerca de 90kg por canal cebo o cebadero': 'Finishing',
'dia tipo mixto': 'Missing',
'dia': 'Missing',
'24 dies produccion de lechones': 'Weaning',
'3 dies produccion de lechones': 'Suckling',
'2 mesos cebo o cebadero': 'Finishing',
'semanas multiplicacion': 'Missing',
'llavores multiplicacion': 'Gilt',
'semana 44 transicion de lechones': 'Finishing',
'lechones primalas produccion de lechones': 'Suckling',
'lechones transicion multiplicacion': 'Nursery',
'lechones <3 semanas': 'Suckling',
'nacidos muertos produccion de ciclo cerrado': 'Sow',
'23 dies de vida produccion de lechones': 'Weaning',
'17-20 semanas multiplicacion': 'Gilt',
'llavores - sortida recria de reproductores': 'Gilt',
'21 dies tipo mixto': 'Weaning',
'lechones 1 y 6 semanas postdestete produccion de lechones': 'Nursery',
'9 semanas seleccion': 'Nursery',
'transicion -': 'Nursery',
'engreix cebo o cebadero': 'Finishing',
'sortida llavores recria de reproductores': 'Gilt',
'verraco adulto centros de recogida de semen': 'Boar',
'cerda 35-37 set vida multiplicacion': 'Gilt',
'indistinta transicion de lechones': 'Nursery',
'9 semanas vida produccion de lechones': 'Nursery',
'6 y 9 semanas de vida': 'Nursery',
'transicion semanas varias produccion de lechones': 'Nursery',
'lechones 1 dia post-destete produccion de lechones': 'Weaning',
'8 semanas transicion de lechones': 'Nursery',
'2, 3, 4 i 5 semanas transicion de lechones': 'Nursery',
'7 meses centros de recogida de semen': 'Boar',
'adulta produccion de lechones': 'Sow',
'paridera produccion de lechones': 'Sow',
'0 y 21 dias -': 'Suckling',
'22 dies produccion de lechones': 'Weaning',
'lechon lactante 1 semana produccion de lechones': 'Suckling',
'2-4 dies de vida produccion de lechones': 'Suckling',
'6 semanas': 'Nursery',
'2 semanas': 'Suckling',
'21 dies produccion de lechones': 'Weaning',
'6-7 semanas transicion de lechones': 'Nursery',
'10 dies post-desmam transicion de lechones': 'Nursery',
'verros produccion de lechones': 'Boar',
'garrins nm produccion de lechones': 'Suckling',
'lechones pre destete (sangre) produccion de lechones': 'Suckling',
'lechones de cerdas primerizas y viejas produccion de lechones': 'Suckling',
'garrins 3d vida tipo mixto': 'Suckling',
'3 s produccion de lechones': 'Weaning',
'primales avortades produccion de lechones': 'Sow',
'primerizas y multiparas produccion de lechones': 'Sow',
'2 meses y medio de cebo cebo o cebadero': 'Finishing',
'entrada llavores seleccion': 'Gilt',
'22 d post destete transicion de lechones': 'Nursery',
'1 mes i mig cebo o cebadero': 'Finishing',
'desmamador produccion de lechones': 'Weaning',
'garrins parideres produccion de lechones': 'Suckling',
'destete (5, 8 i 10 set vida) produccion de lechones': 'Nursery',
'1 dia de vida produccion de lechones': 'Suckling',
'cerda de gestacion (primipara) tipo mixto': 'Sow',
'lechones transicion de lechones': 'Nursery',
'verracos adultos recria de reproductores': 'Boar',
'adulto recria de reproductores': 'Gilt',
'reproductora produccion de lechones': 'Sow',
'0 set (llengues), 4 set (sang) multiplicacion': 'Mixed',
'ritz 1 produccion de lechones': 'Missing',
'llavores recria de reproductores': 'Gilt',
'llavores 40ena tipo mixto': 'Gilt',
'garrins tipo mixto': 'Suckling',
'lechon 7 sem vida': 'Nursery',
'7 setmanes transicion de lechones': 'Nursery',
'2,5 meses de vida cebo o cebadero': 'Finishing',
'cerdas de recria': 'Gilt',
'50 kg': 'Finishing',
'garrins 25d vida produccion de lechones': 'Weaning',
'avorts 50 dies produccion de lechones': 'Sow',
'garrins 25 d vida produccion de lechones': 'Weaning',
'8 set vida transicion de lechones': 'Nursery',
'lechones 3 semanas post-destete -': 'Nursery',
'garrins 1-2 dies produccion de lechones': 'Suckling',
'3 semanas (sangre); 2 dias (hisopos) produccion de lechones': 'Mixed',
'llavores reposicio seleccion': 'Gilt',
'llavores tipo mixto': 'Gilt',
'18 d produccion de lechones': 'Suckling',
'destete (sang) produccion de lechones': 'Weaning',
'lechones de 8 semanas de vida transicion de lechones': 'Nursery',
'lechones de 8 semanas de vida': 'Nursery',
'15 dias produccion de lechones': 'Suckling',
'2-3 sem vida produccion de lechones': 'Suckling',
'3 semanas de vida tipo mixto': 'Weaning',
'1 ano sin datos': 'Other',
'cerdas reposicion': 'Gilt',
'cerdos reposicion produccion de lechones': 'Gilt',
'16 dias produccion de lechones': 'Suckling',
'lechones destetados produccion de lechones': 'Weaning',
'8s transicion de lechones': 'Nursery',
'cerdos engorde cebo o cebadero': 'Finishing',
'recria peculo / lechones maternidad produccion de lechones': 'Mixed',
'truja gestant produccion de lechones': 'Sow',
'verracos adulto centros de recogida de semen': 'Boar',
'llavores 50 kg tipo mixto': 'Gilt',
'destete': 'Weaning',
'lechones lactantes 3 semanas': 'Weaning',
'lechones lactantes 3 semanas produccion de lechones': 'Weaning',
'4-5 setmanes edat transicion de lechones': 'Nursery',
'1 dia produccion de lechones': 'Suckling',
'3,5 meses de vida cebo o cebadero': 'Finishing',
'viejas y primerizas produccion de lechones': 'Sow',
'verraco adulto': 'Boar',
'lechones 1 semana post destete tipo mixto': 'Nursery',
'lechones entre 1 y 2 semanas tipo mixto': 'Suckling',
'entre 11 i 21 setmanes de vida recria de reproductores': 'Gilt',
'3 semanas vida produccion de lechones': 'Weaning',
'garrins 3 set produccion de lechones': 'Weaning',
'garrins 2 set produccion de lechones': 'Suckling',
'reposicion cuarentena multiplicacion': 'Gilt',
'0 y 3 s produccion de lechones': 'Suckling',
'lechon lactacion produccion de lechones': 'Suckling',
'2 dias de vida produccion de lechones': 'Suckling',
'lechon primerizo produccion de lechones': 'Suckling',
'sangre: lechones tipo mixto': 'Suckling',
'5s vida produccion de lechones': 'Suckling',
'5-7 dies de vida multiplicacion': 'Suckling',
'cerdas 85-90kg cebo o cebadero': 'Finishing',
'3 sem vida produccion de lechones': 'Weaning',
'5 dies post destete transicion de lechones': 'Nursery',
'nats morts produccion de lechones': 'Sow',
'23 dies vida produccion de lechones': 'Weaning',
'3 s vida produccion de lechones': 'Weaning',
'garrins 2-3 dies vida seleccion': 'Suckling',
'11 semanas cebo o cebadero': 'Finishing',
'26 dies transicion de lechones': 'Nursery',
'22 dies tipo mixto': 'Weaning',
'cerdas reposicion produccion de lechones': 'Gilt',
'18 semanas de vida cebo o cebadero': 'Finishing',
'lechon transicion 9 semanas produccion de lechones': 'Nursery',
'gestantes 4a sem produccion de lechones': 'Sow',
'lechon tipo mixto': 'Suckling',
'verracos adulto recria de reproductores': 'Boar',
'adultos recria de reproductores': 'Gilt',
'cerdas gestantes / lechones post destete tipo mixto': 'Mixed',
'primalas': 'Gilt',
'control recria de reproductores': 'Gilt',
'lengua: verras // heces: 8 dias (post destete)': 'Mixed',
'8-16 s vida recria de reproductores': 'Gilt',
'lechones 6 semanas post destete produccion de lechones': 'Nursery',
'garrins de 8 setmanes seleccion': 'Nursery',
'0 y 21 dias produccion de lechones': 'Suckling',
'10 setmanes cebo o cebadero': 'Finishing',
'1 mes i mig transicion de lechones': 'Nursery',
'2 semanas produccion de lechones': 'Suckling',
'2 dia post destete transicion de lechones': 'Nursery',
'lechones post-destete produccion de ciclo abierto': 'Nursery',
'lechones-camadas 1r parto 21 dias produccion de lechones': 'Suckling',
'30 dias': 'Nursery',
'sortida cebo o cebadero': 'Finishing',
'futuras reproductoras produccion de lechones': 'Gilt',
'entrada recria multiplicacion': 'Gilt',
'lechones lactantes produccion de lechones': 'Suckling',
'n2-6 set. n4-3 set. transicion de lechones': 'Nursery',
'garri 8 set vida produccion de lechones': 'Nursery',
'garrins produccion de lechones': 'Suckling',
'50kg': 'Finishing',
'recria tipo mixto': 'Gilt',
'lechones primerizas / viejas produccion de lechones': 'Suckling',
'reposicio seleccion': 'Gilt',
'primerizas tipo mixto': 'Sow',
'8 mesos produccion de lechones': 'Finishing',
'cerdas multiparas produccion de lechones': 'Sow',
'8 setmanes de vida produccion de lechones': 'Nursery',
'5 meses': 'Finishing',
'6 mesos recria de reproductores': 'Gilt',
'lechones 10 semanas de edad transicion de lechones': 'Nursery',
'6 semanas cebo o cebadero': 'Nursery',
'2 semanas post-destete / 3 semanas post-destete': 'Nursery',
'truges recria de reproductores': 'Gilt',
'demamador i truges produccion de lechones': 'Mixed',
'lechones mortinatos hijos de cerdas multiparas produccion de lechones': 'Sow',
'lechones mortinatos hijos de cerdas primerizas / multiparas produccion de lechones': 'Sow',
'lechones mortinatos hijode de cerdas primerizas / multiparas produccion de lechones': 'Sow',
'4 setmanes produccion de lechones': 'Weaning',
'sangre lechones de primalas produccion de lechones': 'Suckling',
'muh / prim. tipo mixto': 'Sow',
'cerdas adultas produccion de lechones': 'Sow',
'6 set vida transicion de lechones': 'Nursery',
'maternidad (lenguas) + recria (of) produccion de lechones': 'Mixed',
'6 meses produccion de ciclo cerrado': 'Finishing',
'cebo / 7 semanas': 'Nursery',
'lechones paridera 3 semanas de vida produccion de lechones': 'Weaning',
'lechones 21 dias post-destete produccion de lechones': 'Weaning',
'lechones transicion 9 semanas de vida produccion de lechones': 'Nursery',
'destete, lechones 6-7 semanas de vida': 'Nursery',
'lechones post destete produccion de lechones': 'Nursery',
'sortida produccion de lechones': 'Other',
'primerizas / multiparas produccion de lechones': 'Sow',
'1 dia post destete produccion de lechones': 'Weaning',
'lavores seleccion': 'Gilt',
'0 y 3s produccion de lechones': 'Suckling',
'3s produccion de lechones': 'Weaning',
'50kg recria de reproductores': 'Gilt',
'50kg multiplicacion': 'Gilt',
'1 dia destete produccion de lechones': 'Weaning',
'6 set. vida transicion de lechones': 'Nursery',
'lechones abortados multiplicacion': 'Sow',
'destete multiplicacion': 'Weaning',
'truges i garrins produccion de lechones': 'Mixed',
'7 semanas transicion transicion de lechones': 'Nursery',
'lechones 9 semanas transicion de lechones': 'Nursery',
'7d post destete transicion de lechones': 'Nursery',
'3 semanas, lechon lactacion produccion de lechones': 'Weaning',
'3 semanas; lechon lactacion produccion de lechones': 'Weaning',
'6 set. post destete transicion de lechones': 'Nursery',
'cerdas reposicion cebo o cebadero': 'Gilt',
'cebo 4m y medio cebo o cebadero': 'Finishing',
'lechon con disnea - 3a sem. vida produccion de lechones': 'Weaning',
'lechones 3a semana vida produccion de lechones': 'Weaning',
'truges + garrins tipo mixto': 'Mixed',
'lechones 26 dias de vida tipo mixto': 'Weaning',
'truges (sang); nats morts (pulmons) tipo mixto': 'Mixed',
'garrins parideres': 'Suckling',
'21 d.vida produccion de lechones': 'Weaning',
'garri paridera produccion de lechones': 'Suckling',
'lechon transicion de lechones': 'Nursery',
'lechones 28 dias edad y nacidos muertos produccion de lechones': 'Mixed',
'reposicion 100 kg produccion de lechones': 'Gilt',
'cerdas cuarentena produccion de lechones': 'Gilt',
'reproductoras produccion de lechones': 'Sow',
'adulto': 'Other',
'17 semanas': 'Finishing',
'cerdas de reposicion de 7 meses de edad en cuarentena': 'Gilt',
'18 meses produccion de lechones': 'Other',
'lechones 3-5 dias y lechones destete produccion de lechones': 'Mixed',
'0m1s': 'Suckling',
'lechones destetados tipo mixto': 'Weaning',
'garri parideres produccion de lechones': 'Suckling',
'24 meses produccion de lechones': 'Other',
'vedrraco adulto centros de recogida de semen': 'Boar',
'lechones nacidos vivos produccion de lechones': 'Suckling',
'5 mesos': 'Finishing',
'lechones 3 dias postdestete produccion de lechones': 'Nursery',
'lechones 1 dia postdestete produccion de lechones': 'Weaning',
'25 semanas cebo o cebadero': 'Finishing',
'llavores': 'Gilt',
'12 semanas de vida': 'Finishing',
'transicio, sortida produccion de lechones': 'Nursery',
'lechones 23-25 dias produccion de lechones': 'Weaning',
'lechon lactacion tipo mixto': 'Suckling',
'lechones destetados': 'Weaning',
'lechones transicion aproximadamente 10 kg transicion de lechones': 'Nursery',
'recria, sortida recria de reproductores': 'Gilt',
'destete (5 set vida)': 'Weaning',
'25 dies produccion de lechones': 'Weaning',
'3 setmanes produccion de lechones': 'Weaning',
'cerdas de reposicion produccion de ciclo cerrado': 'Gilt',
'23 dies produccion de lechones': 'Weaning',
'lechones 7,8,9 semanas vida produccion de lechones': 'Nursery',
'lechon primeriza multiplicacion': 'Mixed',
'6-8 meses multiplicacion': 'Gilt',
'garrins (3-4 set) produccion de lechones': 'Weaning',
'verracos adultos': 'Boar',
'3 setm. multiplicacion': 'Weaning',
'24 setmanes vida recria de reproductores': 'Gilt',
'lechones d ecerdas primerizas y viejas produccion de lechones': 'Suckling',
'control entrada recria de reproductores': 'Gilt',
'lechones 26 dias vida produccion de lechones': 'Weaning',
'lechones paridera 3 semanas vida produccion de lechones': 'Weaning',
'cerdas nuliparas produccion de lechones': 'Gilt',
'lechones transicion 9 semanas vida produccion de lechones': 'Nursery',
'cerdas abortadas': 'Sow',
'1 semana multiplicacion': 'Suckling',
'cerdas gestacion produccion de lechones': 'Sow',
'cebo': 'Finishing',
'cerdos 10 semanas de edad': 'Finishing',
'16 sem. cebo o cebadero': 'Finishing',
'1 sem. vida produccion de lechones': 'Suckling',
'25 dies recria de reproductores': 'Weaning',
'7 s transicion de lechones': 'Nursery',
'7 s. transicion de lechones': 'Nursery',
'0 dies vida produccion de lechones': 'Suckling',
'cerdos de 3 semanas de edad produccion de lechones': 'Weaning',
'1 dia vida produccion de lechones': 'Suckling',
'lechones lactacion': 'Suckling',
'garrins 5 d.vida produccion de lechones': 'Suckling',
'garri parideres (7 dies) produccion de lechones': 'Suckling',
'cebadero de ciclo cerrado': 'Finishing',
'cerda de gestacion tipo mixto': 'Sow',
'destete tipo mixto': 'Weaning',
'lechones nacidos muertos': 'Sow',
'lechones de 3, 7 y 10 sem. de vida': 'Mixed',
'cerdas recria produccion de lechones': 'Gilt',
'destete sala 8 (d8) produccion de lechones': 'Weaning',
'destete sala 17 (d17) produccion de lechones': 'Weaning',
'lechon paridera produccion de lechones': 'Suckling',
'5 semanas post-destete // 3 semanas post-destete produccion de lechones': 'Nursery',
'cerdas recria compra pic produccion de lechones': 'Gilt',
'lechon (3 sem) y cerda adulta produccion de lechones': 'Mixed',
'0 y 25 dias produccion de lechones': 'Suckling',
'cebo ibericos': 'Finishing',
'10-18 semanas vida recria de reproductores': 'Gilt',
'8-15 s.vida recria de reproductores': 'Gilt',
'cerda nulipara (30 semanas vida) produccion de lechones': 'Gilt',
'garrins parideres tipo mixto': 'Suckling',
'lechones parto tipo mixto': 'Suckling',
'20 dies de vida produccion de lechones': 'Suckling',
'0 multiplicacion': 'Missing',
'2,5 mesos cebo o cebadero': 'Finishing',
'4 dies vida produccion de lechones': 'Suckling',
'cerdas adultas multiplicacion': 'Sow',
'4 semanas vida produccion de lechones': 'Weaning',
'lechon transicion; 8 semanas de vida transicion de lechones': 'Nursery',
'2m en cebo': 'Finishing',
'lechones mortinatos hijos de cerdas primerizas y multiparas produccion de lechones': 'Sow',
'lechones de cerdas primerizas produccion de lechones': 'Suckling',
'cerda en gestacion produccion de lechones': 'Sow',
'lechones 7 dias vida produccion de lechones': 'Suckling',
'cebo cebadero de ciclo cerrado': 'Finishing',
'nascuts morts tipo mixto': 'Sow',
'truges tipo mixto': 'Sow',
'20 dies produccion de lechones': 'Suckling',
'3 semanas de cebo': 'Finishing',
'10': 'Missing',
'6 meses aprox. multiplicacion': 'Gilt',
'lechones de 5 semanas de vida': 'Nursery',
'diarrea cerdas produccion de lechones': 'Sow',
'8 semanas seleccion': 'Nursery',
'lechones 2 semanas post-destete': 'Nursery',
'transicion cebo o cebadero': 'Nursery',
'lecchon lactacin produccion de lechones': 'Suckling',
'3 dies garrins produccion de lechones': 'Suckling',
'cerdas en gestacion produccion de lechones': 'Sow',
'cerdas': 'Sow',
'cebo de ibericos de 64 dias de estancia': 'Nursery',
'3,5 meses edad cebo o cebadero': 'Finishing',
'cerda 2903 (v9) produccion de lechones': 'Sow',
'23 tipo mixto': 'Missing',
'cerda 5014 (v9) produccion de lechones': 'Sow',
'cerda 5561 (v7) produccion de lechones': 'Sow',
'primerizas seleccion': 'Sow',
'lechones 9-10 semanas edad transicion de lechones': 'Nursery',
'<1 setmana produccion de lechones': 'Suckling',
'28-30 setmanes recria de reproductores': 'Gilt',
'garrins lactants produccion de lechones': 'Suckling',
'cerda 90 dias gestacion': 'Sow',
'entrada multiplicacion': 'Gilt',
'23d produccion de lechones': 'Weaning',
'12 setmanes': 'Finishing',
'15 setmanes': 'Finishing',
'3 setmanes': 'Weaning',
'garrins i adults': 'Mixed',
'entrada recria de reproductores': 'Gilt',
'lechones destete (24d) produccion de lechones': 'Weaning',
'cerda paridera produccion de lechones': 'Sow',
'ritz 2 produccion de lechones': 'Missing',
'23 semanas cebo o cebadero': 'Finishing',
'lechones 9 semanas': 'Nursery',
'lechones 1-2 dia vida produccion de lechones': 'Suckling',
'4, 7 i 10 semanas vida': 'Nursery',
'jovenes/machos produccion de lechones': 'Boar',
'lechones 3 semanas de vida produccion de lechones': 'Weaning',
'1 dia + 1 setmana + 2 setmanes transicion de lechones': 'Mixed',
'25-30kg cebo o cebadero': 'Finishing',
'truges patis produccion de lechones': 'Sow',
'cerdas reposicion 7 meses edad en cuarentena': 'Gilt',
'lechones 4 semanas post destete': 'Nursery',
'llavores cubricio tipo mixto': 'Gilt',
'0': 'Missing',
'lechones maternidad 14 d vida multiplicacion': 'Suckling',
'10 semanas de edad transicion de lechones': 'Nursery',
'reposicion multiplicacion': 'Gilt',
'21 dia tipo mixto': 'Weaning',
'3 setmanes de vida': 'Weaning',
'8 semanas de edad seleccion': 'Nursery',
'lechones 6 y 9 semanas': 'Nursery',
'lechones 1 dia de vida produccion de lechones': 'Suckling',
'lechones paridera tipo mixto': 'Suckling',
'cerdas de reposicion de la nave 3 cebo o cebadero': 'Gilt',
'cerdas reposicion de 5,5 meses produccion de ciclo cerrado': 'Gilt',
'cerdas primerizas gestantes produccion de lechones': 'Sow',
'lechones y intestino lechon 7 dias vida produccion de lechones': 'Suckling',
'6 setmanes': 'Nursery',
'5 semanas': 'Nursery',
'lechon transicion': 'Nursery',
'lechones mortinatos': 'Sow',
'garri nascut (1-2 setmanes) produccion de lechones': 'Suckling',
'7-14 setmanes multiplicacion': 'Mixed',
'2-3 setmanes multiplicacion': 'Suckling',
'cerdas de reposicion de la nave 2 cebo o cebadero': 'Gilt',
'garrins (10 dies de vida); n i r (24 d vida) produccion de lechones': 'Mixed',
'garri destete (5 setmanes) produccion de lechones': 'Weaning',
'2-3 dias de edad produccion de lechones': 'Suckling',
'2a sem vida multiplicacion': 'Suckling',
'destete, 4 sem y 8 sem post-destete': 'Mixed',
'lechones 2-4 dias de vida produccion de lechones': 'Suckling',
'lechon de transicion 6-7 semanas de vida tipo mixto': 'Nursery',
'lactante': 'Suckling',
'2 meses (cebo) cebo o cebadero': 'Finishing',
'transicio': 'Nursery',
'lechones 10 semanas de edad': 'Nursery',
'15 sv': 'Missing',
'multiparas y primalas produccion de lechones': 'Sow',
'30 setmanes': 'Finishing',
'nuliparas 34 semanas de vida produccion de lechones': 'Gilt',
'21d produccion de lechones': 'Weaning',
'21d tipo mixto': 'Weaning',
'destete 10 dies multiplicacion': 'Nursery',
'1d: mostra 1 / 15d: mostra 2 transicion de lechones': 'Mixed',
'3 set i 8 set': 'Mixed',
'recria multiplicacion': 'Gilt',
'lechones maternidad 7-10 dias de vida': 'Suckling',
'2 semanas vida produccion de lechones': 'Suckling',
'llavores reposicio': 'Gilt',
'8 semanas': 'Nursery',
'cerdas jovenes 6-8 meses produccion de lechones': 'Gilt',
'garrins 3 dies vida produccion de lechones': 'Suckling',
'reproductoras produccion de ciclo cerrado': 'Sow',
'lechones cebo o cebadero': 'Finishing',
'2,5 mesos': 'Finishing',
'1-3: cerdas reposicion; 4-9: nacidos muertos produccion de lechones': 'Mixed',
'cerdos 13 sem. de vida cebo o cebadero': 'Finishing',
'abuelas 6 meses': 'Gilt',
'truges / llavores patis produccion de lechones': 'Sow',
'entrada a cebo cebo o cebadero': 'Finishing',
'cerdo de 45kg': 'Finishing',
'lechones paridera 3 semanas produccion de lechones': 'Weaning',
'7 mesos recria de reproductores': 'Gilt',
'garrins nascuts morts produccion de lechones': 'Sow',
'lechones de 3, 6 y 9 semanas': 'Mixed',
'lechones de 10 semanas': 'Nursery',
'mascles centros de recogida de semen': 'Boar',
'2 dies produccion de lechones': 'Suckling',
' "garrins de 28 dies dedat tipo mixto"': 'Weaning',
'lechones 3 semanas (sangre) produccion de lechones': 'Weaning',
'5-6 semanas': 'Nursery',
'10 semanas edad transicion de lechones': 'Nursery',
'7 semanas multiplicacion': 'Nursery',
'6 m': 'Finishing',
'truja + garri produccion de lechones': 'Mixed',
'lechones 30-40kg': 'Finishing',
'cerdas gestantes tipo mixto': 'Sow',
'21 d produccion de lechones': 'Weaning',
'22 d produccion de lechones': 'Weaning',
'23d tipo mixto': 'Weaning',
'lechon lote 2423': 'Suckling',
'28 dies post destete transicion de lechones': 'Nursery',
'cerdas gestantes abortadas produccion de lechones': 'Sow',
'llavores cubricio i patis produccion de lechones': 'Gilt',
'lechones de 3, 6 y 9 semanas de vida': 'Mixed',
'12 semanas': 'Finishing',
'lechones*': 'Suckling',
'recria / cebo 3 produccion de lechones': 'Mixed',
'60 dias vida': 'Finishing',
'garrins destete / parideres': 'Mixed',
'garrins destete': 'Weaning',
'cerdos de 25kg cebo o cebadero': 'Finishing',
'recien entradas recria de reproductores': 'Gilt',
'12 semanas cebo o cebadero': 'Finishing',
'20 semanas cebo o cebadero': 'Finishing',
'transicion tipo mixto': 'Nursery',
'2 meses': 'Nursery',
'cerditas reposicion 8-16 s. vida recria de reproductores': 'Gilt',
'4 set': 'Weaning',
'9 setmanes multiplicacion': 'Nursery',
'25 kg cebo o cebadero': 'Finishing',
'cerdos cebo': 'Finishing',
'lechones 9-10 semanas edad': 'Nursery',
'4-5 semanas postdestete': 'Nursery',
'7 dies': 'Suckling',
'sangre: lechones 1 dia postdestete; pulmon: lechon 4 semanas post destete produccion de lechones': 'Mixed',
'50kg cebo o cebadero': 'Finishing',
'lechones 21 dias 0': 'Weaning',
'lechones 1-2 dia vida': 'Suckling',
'cerdas reposicion de 7 meses de edad en cuarentena': 'Gilt',
'recria 50kg multiplicacion': 'Gilt',
'21 dias 0': 'Weaning',
'llavora (pati/ cubricio) produccion de lechones': 'Gilt',
'4 setmanes transicion de lechones': 'Weaning',
'15-20 kg': 'Nursery',
'garrins paridera 3d vida': 'Suckling',
'15 dies engreix aprox.': 'Finishing',
'lechon 25 dias': 'Weaning',
'22 dies p8-p7 tipo mixto': 'Weaning',
'adultas': 'Sow',
'5 semanas vida transicion de lechones': 'Nursery',
'7 dias 0': 'Suckling',
'garri nascut produccion de lechones': 'Suckling',
'primerizas 3a semana produccion de lechones': 'Sow',
'truja tipo mixto': 'Sow',
'lechones de 9-10 semanas de edad': 'Nursery',
'lechones 18 semanas de vida cebo o cebadero': 'Finishing',
'verracos adulto': 'Boar',
'lechon lactacion': 'Suckling',
'4 semanas de cebo': 'Finishing',
'15-20 kg. cebo o cebadero': 'Nursery',
'lechones (sangres) + semana 13-18:recria y multiparas (lenguas)': 'Mixed',
'10 setmanes': 'Nursery',
'90 dias g produccion de lechones': 'Finishing',
'1,5 m cebo cebo o cebadero': 'Nursery',
'7 semanas vida transicion de lechones': 'Nursery',
'lechones destete cebo o cebadero': 'Weaning',
'6-8 setmanes transicion de lechones': 'Nursery',
'maternidad; 1 semana de vida multiplicacion': 'Suckling',
'1-10:nacimiento / 11-20:destete produccion de lechones': 'Mixed',
'7-8 meses de vida multiplicacion': 'Gilt',
'lechon 3 semanas de vida produccion de lechones': 'Weaning',
'control 2 mesos recria de reproductores': 'Gilt',
'lechones paridera 1 dia de vida produccion de lechones': 'Suckling',
'lechones 3 semanas produccion de lechones': 'Weaning',
'30 semanas de vida produccion de lechones': 'Finishing',
'lechones maternidad': 'Suckling',
'12 semanas de vida cebo o cebadero': 'Finishing',
'primerizas produccion de lechones': 'Gilt',
'garrins lactants seleccion': 'Suckling',
'lechones de 9-10 semanas de vida transicion de lechones': 'Nursery',
'lechones de 9-10 semanas de vida': 'Nursery',
'21-23 semanas cebo o cebadero': 'Finishing',
'15 semanas cebo o cebadero': 'Finishing',
'garrins lactants tipo mixto': 'Suckling',
'lechones paridera 3 semanas vida tipo mixto': 'Weaning',
'111 dies de gestacio multiplicacion': 'Sow',
'14 semanas cebo o cebadero': 'Finishing',
'1 semana vida produccion de lechones': 'Suckling',
'8 semanas de vida produccion de lechones': 'Nursery',
'garri parideres (nascuts) produccion de lechones': 'Suckling',
'primalas y multiparas multiplicacion': 'Sow',
'* -': 'Missing',
'15 dies transicion de lechones': 'Nursery',
'15-20 kgs cebo o cebadero': 'Nursery',
'2 mesos engreix cebo o cebadero': 'Finishing',
'4 dies produccion de lechones': 'Suckling',
'21 d multiplicacion': 'Weaning',
'7, 10, 13 y 16 semanas vida': 'Mixed',
'+6 meses sin datos': 'Finishing',
'20 dies tipo mixto': 'Suckling',
'garri parideres (final) produccion de lechones': 'Suckling',
'2 i 3 set. transicion de lechones': 'Weaning',
'control 3 mesos recria de reproductores': 'Gilt',
'4 dies tipo mixto': 'Suckling',
'lechones paridera 1 semana de vdia produccion de lechones': 'Suckling',
'garrins recria de reproductores': 'Suckling',
'20-10 dies vida produccion de lechones': 'Suckling',
'lechones 3 y 5 semanas multiplicacion': 'Mixed',
'3-4 mesos': 'Finishing',
'lechones 9-10 semanas de vida': 'Nursery',
'lechones 9-10 semanas de vida transicion de lechones': 'Nursery',
'primiparas': 'Sow',
'engreix 60kg': 'Finishing',
'2 mesos': 'Nursery',
'abortos 25 dias produccion de lechones': 'Sow',
'lechones al destete tipo mixto': 'Weaning',
'1 ano': 'Other',
'cerdas abortadas produccion de lechones': 'Sow',
'6 mesos': 'Finishing',
'lechones 3 semanas/ lechones 5 semanas multiplicacion': 'Mixed',
'recria grau produccion de lechones': 'Gilt',
'garrins de parideres de 4 set. de vida tipo mixto': 'Weaning',
'cerdos de 4,5 meses de vida cebo o cebadero': 'Finishing',
'lechones de 9 semanas': 'Nursery',
'1 dia transicion de lechones': 'Weaning',
'0 y 21 dias': 'Suckling',
'lechones nacidos muertos, recogidas en la semana 17 produccion de lechones': 'Sow',
'lechones neonatos': 'Suckling',
'primalas y multiaparas': 'Sow',
'cerdos de engorde; 4 meses de vida cebo o cebadero': 'Finishing',
'1m i 2m': 'Mixed',
'3 setmanes multiplicacion': 'Weaning',
'lechones rota produccion de lechones': 'Suckling',
'ritz produccion de lechones': 'Missing',
'lechones paridera 3 semanas paridera produccion de lechones': 'Weaning',
'1-28: lechones 6 semanas; 29-58: lechones 9 semanas': 'Mixed',
'1 ano centros de recogida de semen': 'Boar',
'5,10 y 150 semanas': 'Mixed',
'verres, 4 mesos de vida de mitja': 'Gilt',
'control llavores': 'Gilt',
'5 setmanes/ 6 setmanes/ 6 mesos (blau)': 'Mixed',
'0 dies': 'Suckling',
'7 setmanes': 'Nursery',
'6 setmanes vida': 'Nursery',
'destete 11 set. vida': 'Finishing',
'lechones paridera 1 semana vida produccion de lechones': 'Suckling',
'* transicion de lechones': 'Nursery',
'1 set. transicion de lechones': 'Nursery',
'garrins paridera produccion de lechones': 'Suckling',
'parideres 1 set.': 'Suckling',
'lechones* transicion de lechones': 'Nursery',
'lechones con 7 semanas de vida transicion de lechones': 'Nursery',
'1 setmana multiplicacion': 'Suckling',
'1 setmana': 'Suckling',
'garri produccion de lechones': 'Suckling',
'recrias': 'Gilt',
'primerizas multiplicacion': 'Sow',
'cerdos 6 semes': 'Finishing',
'verracos nuevos centros de recogida de semen': 'Boar',
'sangre madres multiplicacion': 'Sow',
'3 mesos recria de reproductores': 'Gilt',
'lechones de 8 semanas de vida produccion de lechones': 'Nursery',
'3-6-9 semanas de vida': 'Mixed',
'sortides recria de reproductores': 'Gilt',
'5 mesos recria de reproductores': 'Gilt',
'reria.': 'Gilt',
'10 centros de recogida de semen': 'Boar',
'cerdos de cebo': 'Finishing',
"garrins de 28 dies d'edat tipo mixto": 'Weaning',
"15-21 dies d'entrada a l'engreix cebo o cebadero": 'Finishing',


}

df1['PigStage'] = df1['AnimalType'].map(PigStageMapping).fillna('Missing')

print(df1['PigStage'].unique())

['Missing' 'Mixed' 'Nursery' 'Boar' 'Finishing' 'Gilt' 'Weaning'
 'Suckling' 'Sow' 'Other']


In [99]:
# Province Mapping and annotations
# Province to Region (Autonomous Community) Mapping
ProvinceMapping = {
   '0': 'Missing',
    '-': 'Missing',
    '?': 'Missing',
    'albacete': 'Albacete',
    'almeria': 'Almeria',
    'badajoz': 'Badajoz',
    'barcelona': 'Barcelona',
    'caceres': 'Caceres',
    'castello': 'Castellon',
    'castellon': 'Castellon',
    'cordoba': 'Cordoba',
    'cuenca': 'Cuenca',
    'gerona': 'Girona',
    'girona': 'Girona',
    'granada': 'Granada',
    'granada?': 'Missing',
    'huesca': 'Huesca',
    'huesca?': 'Missing',
    'la rioja': 'LaRioja',
    'las palmas': 'LasPalmas',
    'lerida': 'Lleida',
    'lleida': 'Lleida',
    'lleida?': 'Missing',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'murcia?': 'Missing',
    'navarra': 'Navarre',
    'None': 'Missing',
    'orense': 'Ourense',
    'ourense': 'Ourense',
    'salamanca': 'Salamanca',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'soria': 'Soria',
    'tarragona': 'Tarragona',
    'tarragona?': 'Missing',
    'teruel': 'Teruel',
    'teruel?': 'Missing',
    'toledo': 'Toledo',
    'treuel': 'Teruel',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}


df1["Province"] = df1["Province"].map(ProvinceMapping).fillna("Missing")

# Convert 'Date' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
# Format 'Date' column as 'DD-MM-YYYY'
#df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')


In [100]:
# Drop intermediate columns if needed
df1.drop(['AnimalType'], axis=1, inplace=True)

In [101]:
df1.head(50)

,SampleNumber,Pathogen,Date,Result,Province,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,1,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
1,2,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
2,3,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
3,4,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
4,5,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
5,6,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
6,7,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
7,8,Actinobacillus pleuropneumoniae,2024-12-31,1,Missing,Other,PCR,Spain,Meat,1,Missing
8,9,Porcine reproductive and respiratory syndrome ...,2024-12-31,0,Tarragona,Mixed,PCR,Spain,Meat,1,Mixed
9,10,Porcine reproductive and respiratory syndrome ...,2024-12-31,0,Tarragona,Mixed,PCR,Spain,Meat,1,Mixed


In [102]:
print(df1['Pathogen'].unique())

['Actinobacillus pleuropneumoniae'
 'Porcine reproductive and respiratory syndrome virus'
 'Porcine epidemic diarrhea virus' 'Swine influenza virus'
 'Brachyspira hyodysenteriae' 'Mycoplasma hyopneumoniae'
 'Lawsonia intracellularis' 'Brachyspira spp']


In [103]:
print(df1['Result'].unique())

['1' '0' 'Missing']


In [104]:
print(df1['SampleType'].unique())

['Other' 'Mixed' 'Blood' 'Swab' 'Fluid' 'Tissue' 'Faeces']


In [105]:
print(df1['PigStage'].unique())

['Missing' 'Mixed' 'Nursery' 'Boar' 'Finishing' 'Gilt' 'Weaning'
 'Suckling' 'Sow' 'Other']


In [106]:
print(df1['Province'].unique())

['Missing' 'Tarragona' 'Huesca' 'Zaragoza' 'Barcelona' 'Lleida' 'Girona'
 'Granada' 'Valencia' 'Sevilla' 'Almeria' 'Murcia' 'Badajoz' 'Soria'
 'Segovia' 'Navarre' 'Castellon' 'Teruel' 'LaRioja' 'Cuenca' 'Toledo'
 'Caceres' 'Zamora' 'Ourense' 'Cordoba' 'Albacete' 'Malaga' 'Salamanca'
 'Valladolid' 'LasPalmas']


In [107]:
df1.to_csv('output/PigLab1_2.csv', index=False)

### Join both datasets

In [108]:
df_all = pd.concat([df, df1], ignore_index = True)

print(df_all['Pathogen'].unique())

['Swine influenza virus' 'Lawsonia intracellularis'
 'Mycoplasma hyopneumoniae' 'Actinobacillus pleuropneumoniae'
 'Porcine reproductive and respiratory syndrome virus'
 'Porcine epidemic diarrhea virus' 'Brachyspira hyodysenteriae'
 'Brachyspira spp']


In [109]:
df_all.to_csv('output/PigLab1_update20250109.csv', index=False)

### Step 02: Create an RDF graph and namespaces.

In [110]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [111]:
 # for slugging local names or avoid spaces in names to create URIs along with label names

import re
import urllib.parse

def _safe_local_name(v):
    v = str(v).strip()
    # RFC3986 unreserved: A-Z a-z 0-9 - . _ ~
    v = re.sub(r'[^A-Za-z0-9\-._~]', '_', v)
    v = re.sub(r'_+', '_', v)  # collapse repeated underscores (optional)
    return v

In [112]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
##................................><.........................................................................
##Note: spaces in URIs are not OK. RDFLib (and the RDF specs) require valid URIs; a space makes them invalid. 
##If there is space in names then got errors in the Turtle serialization.
##................................><.........................................................................
for index, row in df_all.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))

    
    #if "Pathogen" in row and row["Pathogen"]:    # Making little changes to pathogen URI so that it will not create error
        #Pathogen = row["Pathogen"]
        #Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        #g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        #g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        #description = "An individual representing Pig pathogen."
        #g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))

    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = str(row["Pathogen"]).strip()
        Pathogen_local = _safe_local_name(Pathogen)  # e.g., "Swine_influenza_virus"
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen_local}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        g.add((Pathogen_uri, RDFS.label, Literal(Pathogen)))  # human-readable conventional label, e.g., "Swine influenza virus" same thing we can do with other properties like breed and diagnostic test etc. if we have sapce

        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))

    if "Lab_Reference" in row and row["Lab_Reference"]:  #Adding that code
        Lab_Reference = row["Lab_Reference"]
        Lab_Reference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Lab_Reference}")
        g.add((PigSample_uri, LHO.hasLab_Reference, Lab_Reference_uri))
        g.add((Lab_Reference_uri, RDF.type, LHO.Lab_Reference))
        description = "An individual representing from which lab refence it belongs."
        g.add((Lab_Reference_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
#turtle_data = g.serialize(format="turtle")
#print(turtle_data)

<Graph identifier=Nc25079de790d4859aa9e7385bdfdbe34 (<class 'rdflib.graph.Graph'>)>

### Step 4: Load the RDF data and ontology into a Panda DataFrame: 

In [113]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=Nc25079de790d4859aa9e7385bdfdbe34 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [114]:
query = """
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO:   <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT
  (STRAFTER(STR(?Sample), "#") AS ?SampleNumber)
  (STRAFTER(STR(?Province), "#") AS ?Province)
  (?FlooredDate AS ?FlooredDate)
  (COALESCE(STR(?PathogenLabel), REPLACE(STRAFTER(STR(?Pathogen), "#"), "_", " ")) AS ?Pathogen)
  (STRAFTER(STR(?SampleResult), "#") AS ?Result)
  (STRAFTER(STR(?SampleType), "#") AS ?SampleType)
  (STRAFTER(STR(?DiagnosticTest), "#") AS ?DiagnosticTest)
  (STRAFTER(STR(?Country), "#") AS ?Country)
  (STRAFTER(STR(?Breed), "#") AS ?Breed)
  (STRAFTER(STR(?Lab_Reference), "#") AS ?Lab_Reference)
  (STRAFTER(STR(?PigProductionStages), "#") AS ?PigStage)
  (STRAFTER(STR(?AutonomousCommunity), "#") AS ?AutonomousCommunity)
  (?MonthNum AS ?Month)
  (?YearNum  AS ?Year)
WHERE {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasDate ?Date .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasLab_Reference ?Lab_Reference .
  OPTIONAL { ?Sample LHO:hasAutonomousCommunity ?AutonomousCommunity . }

  # Pathogen label only (prefer rdfs:label/skos:prefLabel; prefer en or no-lang)
  OPTIONAL {
    ?Pathogen (rdfs:label|skos:prefLabel) ?PathogenLabel .
    FILTER(LANG(?PathogenLabel) = "" || LANGMATCHES(LANG(?PathogenLabel), "en"))
  }

  # ---- Date parsing (supports DD/MM/YYYY and YYYY-MM-DD) ----
  BIND( STRAFTER(STR(?Date), "#") AS ?DateStr )
  BIND( ?DateStr AS ?FlooredDate )

  # month
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 4, 2),   # DD/MM/YYYY -> MM is positions 4-5
         SUBSTR(?DateStr, 6, 2)    # YYYY-MM-DD -> MM is positions 6-7
      )
    ) AS ?MonthNum
  )

  # year
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 7, 4),   # DD/MM/YYYY -> YYYY starts at 7
         SUBSTR(?DateStr, 1, 4)    # YYYY-MM-DD -> YYYY is positions 1-4
      )
    ) AS ?YearNum
  )
}
"""
results = g.query(query)

data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "SampleNumber","Province","FlooredDate","Pathogen","Result","SampleType",
    "DiagnosticTest","Country","Breed","LabReference","PigStage",
    "AutonomousCommunity","Month","Year"
])


In [115]:
df

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Missing,2021-01-05,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Missing,None,1,2021
1,Lab1PigSample_1,Huesca,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Sow,None,1,2021
2,Lab1PigSample_2,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing,None,1,2021
3,Lab1PigSample_3,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing,None,1,2021
4,Lab1PigSample_4,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing,None,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14770,Lab1PigSample_14770,Huesca,2024-01-09,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,1,Missing,None,1,2024
14771,Lab1PigSample_14771,Huesca,2024-01-09,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,1,Boar,None,1,2024
14772,Lab1PigSample_14772,Huesca,2024-01-09,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,1,Boar,None,1,2024
14773,Lab1PigSample_14773,Lleida,2024-01-09,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,1,Boar,None,1,2024


In [116]:
# Save Results: save to CSV
import os
os.makedirs("output", exist_ok=True)

df.to_csv("output/PigLab1_query_results.csv", index=False, encoding="utf-8")
print("Saved: output/PigLab1_query_results_baromterepig.csv")

Saved: output/PigLab1_query_results_baromterepig.csv
